In [35]:
import os
import random

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import cv2
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
import wandb

from ipywidgets import IntSlider, interact
from matplotlib import animation, rc
from matplotlib.patches import PathPatch, Rectangle
from matplotlib.path import Path
from scipy import ndimage
from scipy.ndimage import zoom
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from keras import Input, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import (BatchNormalization, Conv3D, Dense,
                                     Dropout, GlobalAveragePooling3D,
                                     MaxPool3D)
from keras.optimizers import Adam
from wandb.keras import WandbCallback

import keras_applications_3d as ka3d
from keras_applications_3d import vgg16, vgg19, resnet, resnet_v2, densenet

Data loading and preprocessing

In [15]:
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))
    return volume.astype('float32')

def resize_volume(img, desired_width=128, desired_height=128, desired_depth=64):
    """Resize the volume"""
    # Compute zoom factors
    width_factor = desired_width / img.shape[0]
    height_factor = desired_height / img.shape[1]
    depth_factor = desired_depth / img.shape[-1]
    # Rotate volume by 90 degrees
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize the volume using spline interpolated zoom (SIZ)
    img = zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    return volume

Class Distribution

In [32]:
# TODO - functions to load data and labels

# print(f'Number of 
nb_classes = 2

Build Train and Validation Datasets

In [ ]:
# TODO - build train, validation, test sets

Data Augmentation

Les données originales sont représentées sous la forme de tenseurs tridimensionnels avec des dimensions (samples, height, width, depth). Cependant, afin de permettre l'utilisation de convolutions 3D sur ces données, nous devons ajouter une dimension supplémentaire à nos tenseurs. Cette dimension est ajoutée à l'axe 4, qui représente le nombre de canaux. En général quand on traite des images 2D en couleurs on a 3 canaux pour le RGB, ici on traite des images 3D en pseudo NB. Donc en ajoutant une dimension, les données sont transformées en tenseurs de forme (samples, height, width, depth, 1)

In [ ]:
# TODO - Add a dimension to the data to make it 4D (for the channel)

In [ ]:
# TODO - Add data augmentation (rotation, translation, scaling, etc.)

In [ ]:
# TODO - Build train_generator, validation_generator, test_generator

Augmented CT Scan Visualization

In [ ]:
# TODO - Check the shape of the data

In [ ]:
# TODO - MRI images are composed by many slices, build a montage of the them

3D Convolutional Neural Network

In [41]:
def get_model(height=113, width=137, depth=113):
    """Build a 3D convolutional neural network model."""
    
    net = vgg19.VGG19(
        input_shape=(height, width, depth, 1),
        include_top=False,
        weights=None,
        classes=nb_classes,
        base_channel=16
    )
    
    x = net.layers[-1].output
    x = GlobalAveragePooling3D()(x)    
    x = Dense(units=128, activation='relu')(x)
    x = Dense(units=64, activation='relu')(x)
    outputs = Dense(units=nb_classes, activation='softmax')(x)
    
    return Model(net.inputs, outputs, name='3D-ResNet50')


# Build model.
model = get_model(height=113, width=137, depth=113)
model.summary()

Model: "3D-ResNet50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 113, 137, 113,    0         
                             1)]                                 
                                                                 
 block1_conv1 (Conv3D)       (None, 113, 137, 113, 1   448       
                             6)                                  
                                                                 
 block1_conv2 (Conv3D)       (None, 113, 137, 113, 1   6928      
                             6)                                  
                                                                 
 block1_pool (MaxPooling3D)  (None, 56, 68, 56, 16)    0         
                                                                 
 block2_conv1 (Conv3D)       (None, 56, 68, 56, 32)    13856     
                                                       

Model training

In [46]:
# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-4),
    metrics=['acc'],
)



# Train the model, doing validation at the end of each epoch
# model.fit()

Visualizing Training History

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 3))
ax = ax.ravel()

for i, metric in enumerate(['acc', 'loss']):
    ax[i].plot(model.history.history[metric])
    ax[i].plot(model.history.history[f'val_{metric}'])
    ax[i].set_title(f'Model {metric}')
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(metric)
    ax[i].legend(['train', 'val'])

Model Evaluation

In [ ]:
# TODO - Evaluate the model on the test set